In [1]:
import numpy as np
from keras import models, layers, callbacks
from scipy.io import loadmat
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
train_mat = loadmat('data/train_32x32.mat')
test_mat = loadmat('data/test_32x32.mat')

#Format into np array of float pixels and 1d array of ints for labels
train_imgs = np.array(train_mat['X']).astype('float64')
train_lbls = np.array(train_mat['y']).flatten()
test_imgs = np.array(test_mat['X']).astype('float64')
test_lbls = np.array(test_mat['y']).flatten()

#Normalize to 0..1
train_imgs = train_imgs/255.0
test_imgs = test_imgs/255.0

In [ ]:
train_imgs = np.moveaxis(train_imgs, -1, 0)
test_imgs = np.moveaxis(test_imgs, -1, 0)

In [ ]:
plt.imshow(train_imgs[2241])
plt.show()

print('Label: ', train_lbls[2241])

In [ ]:
#Binarize labels
binarizer = LabelBinarizer()
train_lbls = binarizer.fit_transform(train_lbls)
test_lbls = binarizer.fit_transform(test_lbls)

In [2]:
X_trn, X_tst, y_trn, y_tst = train_test_split(train_imgs, train_lbls, test_size=0.2, random_state=0)
print(X_trn.shape, X_tst.shape, y_trn.shape, y_tst.shape)

NameError: name 'train_test_split' is not defined

In [ ]:
#CNN Model

model = models.Sequential(
    [
        layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid'),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
        layers.Dropout(0.3),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')        
    ]
)

In [ ]:
#utility functions
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1)
red_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.00001, verbose=1)

In [ ]:
#Define parameters for applying random transformations to train data
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#Generate batches of transformed image data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=X_trn,
    target_size=(32, 32),
    batch_size=256,
    shuffle=True,
    class_mode='categorical')

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, y_trn, batch_size=256, epochs=15, validation_data=(X_tst, y_tst), callbacks=[early_stopping, red_lr])